In [1]:
import numpy as np
import gzip
import cPickle

In [2]:
import six

In [3]:
DATA_FILENAME = '/Users/merlin/Documents/Neural_Net_Newbies/data/mnist.pkl.gz'

### im2col

In [4]:
f = gzip.open(DATA_FILENAME,'rb')
train_set, valid_set, test_set = cPickle.load(f)
f.close()

In [5]:
train_set_x, train_set_y = train_set

In [89]:
oneEx = train_set_x[0]

In [92]:
img = oneEx.reshape(1,1,28,28)

In [93]:
label = train_set_y[0]

In [94]:
n, c, h, w = img.shape
# num examples, channels in, height, width

In [95]:
ph = 1
pw = 1
sy = 3
sx = 3
pval = False

In [96]:
img = np.pad(img,((0, 0), (0, 0), (ph, ph + sy - 1), (pw, pw + sx - 1)),
mode='constant', constant_values=(pval,))

In [97]:
img.shape

(1, 1, 32, 32)

In [98]:
kh=9
kw=9

In [99]:
out_h = (h-kh+ph*2)/sy + 1

In [100]:
out_h

8

In [101]:
out_w=out_h

In [102]:
col = np.ndarray((n,c,kh,kw,out_h,out_w), dtype=img.dtype)

In [103]:
col.shape

(1, 1, 9, 9, 8, 8)

In [104]:
for i in six.moves.range(kh): 
    i_lim = i + sy * out_h
    for j in six.moves.range(kw):
        j_lim = j + sx * out_w
        get = img[:, :, i:i_lim:sy, j:j_lim:sx]
        col_assign = col[:, :, i, j, :, :]
        col[:, :, i, j, :, :] = get

In [105]:
col_assign.shape

(1, 1, 8, 8)

In [106]:
print i_lim, j_lim

32 32


In [107]:
get.shape

(1, 1, 8, 8)

In [108]:
col.shape

(1, 1, 9, 9, 8, 8)

In [109]:
weights = np.random.rand(20,1,9,9)

In [110]:
weights.shape

(20, 1, 9, 9)

In [111]:
result = np.tensordot(col, weights, ([1, 2, 3], [1, 2, 3]))

In [112]:
result.shape

(1, 8, 8, 20)

In [113]:
col.shape

(1, 1, 9, 9, 8, 8)

In [114]:
bias = np.random.rand(20,)

In [115]:
bias.shape

(20,)

In [116]:
result += bias

In [117]:
result.shape

(1, 8, 8, 20)

In [118]:
featureMapWidth = (w-kw+pw*2)/sx + 1
featureMapHeight = (h-kh+ph*2)/sy + 1 

In [119]:
featureMapHeight

8

### Checking activations & backprop - CNN

In [89]:
np.set_printoptions(threshold=np.nan)

In [90]:
new_val = np.array([[[[1,1,1,1,1,1,1,1],[2,2,2,2,2,2,2,2],[3,3,3,3,3,3,3,3],[4,4,4,4,4,4,4,4],[1,1,1,1,1,1,1,1],[2,2,2,2,2,2,2,2],[3,3,3,3,3,3,3,3],[4,4,4,4,4,4,4,4]]]])

In [91]:
new_val.ravel()

array([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4])

In [92]:
# can't see rollign if use smaller version
#new_val = np.array([[[[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]]])

In [93]:
new_val.shape

(1, 1, 8, 8)

In [94]:
n, c, h, w = new_val.shape
kh=2
kw=2
ph = 0
pw = 0
sy = 2
sx = 2
depth = 2

In [95]:
def im2col(img, kh, kw, sy, sx, ph, pw, pval=0, cover_all=False):
    n, c, h, w = img.shape
    featureMapWidth = (w-kw+pw*2)/sx + 1
    featureMapHeight = (h-kh+ph*2)/sy + 1 

    img = np.pad(img, ((0, 0), (0, 0), (ph, ph + sy - 1), (pw, pw + sx - 1)),
                    mode='constant', constant_values=(pval,))
    col = np.ndarray((n, c, kh, kw, featureMapHeight, featureMapWidth), dtype=img.dtype)

    for i in six.moves.range(kh):
        i_lim = i + sy * featureMapHeight
        for j in six.moves.range(kw):
            j_lim = j + sx * featureMapWidth
            col[:, :, i, j, :, :] = img[:, :, i:i_lim:sy, j:j_lim:sx]
    return col

def col2im(col, sy, sx, ph, pw, h, w):
    n, c, kh, kw, out_h, out_w = col.shape

    img = np.zeros((n, c, h + 2 * ph + sy - 1, w + 2 * pw + sx - 1), dtype=col.dtype)
    for i in six.moves.range(kh):
        i_lim = i + sy * out_h
        for j in six.moves.range(kw):
            j_lim = j + sx * out_w
            img[:, :, i:i_lim:sy, j:j_lim:sx] += col[:, :, i, j, :, :]

    return img[:, :, ph:h + ph, pw:w + pw]

In [96]:
# W = np.random.rand(depth, c, kh, kw)
# b = np.random.rand(depth,)

W = np.array([[[[ 0.5,  0.5 ], [ 0.5,  0.5]]], [[[ 0.5,  0.5], [ 0.5,  0.5]]]])
b = np.array([ 1,  1])

In [97]:
W.shape

(2, 1, 2, 2)

In [98]:
featureMapWidth = (w-kw+pw*2)/sx + 1
featureMapHeight = (h-kh+ph*2)/sy + 1 

In [99]:
padded = np.pad(new_val , ((0, 0), (0, 0), (ph, ph + sy - 1), (pw, pw + sx - 1)),
                    mode='constant')

In [100]:
padded.shape

(1, 1, 9, 9)

In [101]:
padded

array([[[[1, 1, 1, 1, 1, 1, 1, 1, 0],
         [2, 2, 2, 2, 2, 2, 2, 2, 0],
         [3, 3, 3, 3, 3, 3, 3, 3, 0],
         [4, 4, 4, 4, 4, 4, 4, 4, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0],
         [2, 2, 2, 2, 2, 2, 2, 2, 0],
         [3, 3, 3, 3, 3, 3, 3, 3, 0],
         [4, 4, 4, 4, 4, 4, 4, 4, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0]]]])

In [102]:
col = np.ndarray((n, c, kh, kw, featureMapHeight, featureMapWidth))

In [103]:
col.shape

(1, 1, 2, 2, 4, 4)

In [104]:
for i in six.moves.range(kh):
    i_lim = i + sy * featureMapHeight
    for j in six.moves.range(kw):
        j_lim = j + sx * featureMapWidth
        col[:, :, i, j, :, :] = new_val[:, :, i:i_lim:sy, j:j_lim:sx]

In [105]:
col 

array([[[[[[ 1.,  1.,  1.,  1.],
           [ 3.,  3.,  3.,  3.],
           [ 1.,  1.,  1.,  1.],
           [ 3.,  3.,  3.,  3.]],

          [[ 1.,  1.,  1.,  1.],
           [ 3.,  3.,  3.,  3.],
           [ 1.,  1.,  1.,  1.],
           [ 3.,  3.,  3.,  3.]]],


         [[[ 2.,  2.,  2.,  2.],
           [ 4.,  4.,  4.,  4.],
           [ 2.,  2.,  2.,  2.],
           [ 4.,  4.,  4.,  4.]],

          [[ 2.,  2.,  2.,  2.],
           [ 4.,  4.,  4.,  4.],
           [ 2.,  2.,  2.,  2.],
           [ 4.,  4.,  4.,  4.]]]]]])

In [106]:
col = im2col(new_val, kh, kw, sy, sx, ph, pw)

In [107]:
col #convolved

array([[[[[[1, 1, 1, 1],
           [3, 3, 3, 3],
           [1, 1, 1, 1],
           [3, 3, 3, 3]],

          [[1, 1, 1, 1],
           [3, 3, 3, 3],
           [1, 1, 1, 1],
           [3, 3, 3, 3]]],


         [[[2, 2, 2, 2],
           [4, 4, 4, 4],
           [2, 2, 2, 2],
           [4, 4, 4, 4]],

          [[2, 2, 2, 2],
           [4, 4, 4, 4],
           [2, 2, 2, 2],
           [4, 4, 4, 4]]]]]])

In [108]:
col.shape

(1, 1, 2, 2, 4, 4)

In [109]:
y = np.tensordot(col, W, ([1, 2, 3], [1, 2, 3]))

In [110]:
W.shape

(2, 1, 2, 2)

In [111]:
y

array([[[[ 3.,  3.],
         [ 3.,  3.],
         [ 3.,  3.],
         [ 3.,  3.]],

        [[ 7.,  7.],
         [ 7.,  7.],
         [ 7.,  7.],
         [ 7.,  7.]],

        [[ 3.,  3.],
         [ 3.,  3.],
         [ 3.,  3.],
         [ 3.,  3.]],

        [[ 7.,  7.],
         [ 7.,  7.],
         [ 7.,  7.],
         [ 7.,  7.]]]])

In [112]:
y.shape

(1, 4, 4, 2)

In [113]:
b

array([1, 1])

In [114]:
if b is not None: new_y = y+ b

In [115]:
new_y

array([[[[ 4.,  4.],
         [ 4.,  4.],
         [ 4.,  4.],
         [ 4.,  4.]],

        [[ 8.,  8.],
         [ 8.,  8.],
         [ 8.,  8.],
         [ 8.,  8.]],

        [[ 4.,  4.],
         [ 4.,  4.],
         [ 4.,  4.],
         [ 4.,  4.]],

        [[ 8.,  8.],
         [ 8.,  8.],
         [ 8.,  8.],
         [ 8.,  8.]]]])

In [116]:
new_y.ravel()

array([ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  8.,  8.,  8.,  8.,  8.,
        8.,  8.,  8.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  8.,  8.,
        8.,  8.,  8.,  8.,  8.,  8.])

In [117]:
new_y.shape

(1, 4, 4, 2)

In [118]:
y2 = np.rollaxis(new_y, 3, 1)

In [119]:
y2.ravel()

array([ 4.,  4.,  4.,  4.,  8.,  8.,  8.,  8.,  4.,  4.,  4.,  4.,  8.,
        8.,  8.,  8.,  4.,  4.,  4.,  4.,  8.,  8.,  8.,  8.,  4.,  4.,
        4.,  4.,  8.,  8.,  8.,  8.])

In [120]:
y2.shape

(1, 2, 4, 4)

In [121]:
y2.shape

(1, 2, 4, 4)

In [122]:
y2

array([[[[ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.],
         [ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.]],

        [[ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.],
         [ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.]]]])

In [123]:
def sigmoid(x):
        return 1 / (1 + np.exp(-x))

In [124]:
sigmoid = np.vectorize(sigmoid)

In [125]:
r = sigmoid(y2)

In [126]:
r.shape

(1, 2, 4, 4)

#### Backprop

In [368]:
def col2im_cpu(col, sy, sx, ph, pw, h, w):
    n, c, kh, kw, out_h, out_w = col.shape

    img = np.zeros((n, c, h + 2 * ph + sy - 1, w + 2 * pw + sx - 1), dtype=col.dtype)
    for i in six.moves.range(kh):
        i_lim = i + sy * out_h
        for j in six.moves.range(kw):
            j_lim = j + sx * out_w
            img[:, :, i:i_lim:sy, j:j_lim:sx] += col[:, :, i, j, :, :]

    return img[:, :, ph:h + ph, pw:w + pw]

def backward_cpu(x, gy):
    gbias = np.array([1, 1])
    gWeight = np.array([[[[ 0.5,  0.5 ], [ 0.5,  0.5]]], [[[ 0.5,  0.5], [ 0.5,  0.5]]]])
    if gbias is not None:
        gbias += gy.sum(axis=(0, 2, 3))
    gWeight += np.tensordot(gy, col, ([0, 2, 3], [0, 4, 5]))
    gcol = np.tensordot(W, gy, (0, 1))
    gcol = np.rollaxis(gcol, 3)

    h, w = x.shape[2:]
    return col2im(gcol, sy, sx, ph, pw, h, w)

def calcDelta(epsilon):
    z = np.tensordot(col, W, ([1, 2, 3], [1, 2, 3]))
    z += b
    z = np.rollaxis(z, 3, 1)
    d = derivative(expit, z)
    return epsilon * d

def backprop(epsilon):
    # get z, activationDerivative, * to the epsilon for the delta
    delta = calcDelta(epsilon)
    
    weight_gradient = np.tensordot(delta, col, ([0, 2, 3], [0, 4, 5]))
    gb = delta.sum(axis=(0, 2, 3))
    nextEpsilon = np.tensordot(W, delta, (0, 1))
    nextEpsilon = np.rollaxis(nextEpsilon, 3)
    
    
    h, w = new_val.shape[2:]
    return col2im(nextEpsilon, sy, sx, ph, pw, h, w)

In [314]:
r2 = backprop(epsilon)

In [372]:
backward_cpu(new_val, an)

array([[[[ 0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.00039383,
           0.00039383,  0.        ,  0.        ],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.00039383,
           0.00039383,  0.        ,  0.        ],
         [ 0.02036651,  0.02036651,  0.02036651,  0.02036651,  0.02036651,
           0.02036651,  0.        ,  0.        ],
         [ 0.02036651,  0.02036651,  0.02036651,  0.02036651,  0.02036651,
           0.02036651,  0.        ,  0.        ],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.00039383,
           0.00039383,  0.        ,  0.        ],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.00039383,
           0.00039383,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ]]]

In [128]:
reverse_input = y2

In [129]:
# delta
gy = np.array([[[[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ]], [[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ],[ 2, 2, 2, 2 ]]]])
gb = np.array([1,1])

In [130]:
gy.shape

(1, 2, 4, 4)

In [131]:
gy[0].sum(axis = (1,2)) # same as gy.sum(axis = (0,2,3))

array([32, 32])

In [132]:
gb = gy[0].sum(axis = (1,2)) + gb

In [133]:
gb

array([33, 33])

In [134]:
gW = np.tensordot(gy, col, ([0, 2, 3], [0, 4, 5]))

In [135]:
gW

array([[[[64, 64],
         [96, 96]]],


       [[[64, 64],
         [96, 96]]]])

In [136]:
gW.shape

(2, 1, 2, 2)

In [137]:
W.shape

(2, 1, 2, 2)

In [138]:
gcol = np.tensordot(W, gy, (0, 1))

In [139]:
gcol.shape

(1, 2, 2, 1, 4, 4)

In [141]:
gcol = np.rollaxis(gcol, 3)

In [142]:
gcol.shape

(1, 1, 2, 2, 4, 4)

##### preOutput

In [143]:
def deriv_sigmoid(x):
    v = np.exp(x) / np.power(1+np.exp(x),2.0)
    v = x * (1-x)
    return v


In [176]:
def sigmoid_grad(x):
    ex = np.exp(-x)
    y = ex / (1 + ex)**2
    return y

In [275]:
def sigmoid_grad_sech2(x):
    y = (0.5 / np.cosh(0.5*x))**2
    return y

sigmoid(x)*(1-sigmoid(x))

In [144]:
deriv_sigmoid = np.vectorize(deriv_sigmoid)

In [177]:
sigmoid_grad_vec = np.vectorize(sigmoid_grad)

In [276]:
sig_grad_vec2 = np.vectorize(sigmoid_grad_sech2)

In [145]:
z = np.tensordot(col, W, ([1, 2, 3], [1, 2, 3]))

In [146]:
z.shape

(1, 4, 4, 2)

In [147]:
z+=b

In [161]:
z

array([[[[ 4.,  4.],
         [ 4.,  4.],
         [ 4.,  4.],
         [ 4.,  4.]],

        [[ 8.,  8.],
         [ 8.,  8.],
         [ 8.,  8.],
         [ 8.,  8.]],

        [[ 4.,  4.],
         [ 4.,  4.],
         [ 4.,  4.],
         [ 4.,  4.]],

        [[ 8.,  8.],
         [ 8.,  8.],
         [ 8.,  8.],
         [ 8.,  8.]]]])

In [222]:
z = np.tensordot(col, W, ([1, 2, 3], [1, 2, 3]))
z += b
z = np.rollaxis(z,3,1)

##### delta

In [278]:
z

array([[[[ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.],
         [ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.]],

        [[ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.],
         [ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.]]]])

In [297]:
# delta
d = derivative(expit, z)
an = epsilon * d

In [280]:
d

array([[[[ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524],
         [ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524]],

        [[ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524],
         [ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524]]]])

In [291]:
sig_grad_vec2(z)

array([[[[ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524],
         [ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524]],

        [[ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524],
         [ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524]]]])

In [293]:
from scipy.special import expit
expit(z)*(1-expit(z))

array([[[[ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524],
         [ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524]],

        [[ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524],
         [ 0.01766271,  0.01766271,  0.01766271,  0.01766271],
         [ 0.00033524,  0.00033524,  0.00033524,  0.00033524]]]])

In [343]:
from scipy.misc import derivative
derivative(expit, z)

array([[[[ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383],
         [ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383]],

        [[ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383],
         [ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383]]]])

In [298]:
an.shape

(1, 2, 4, 4)

In [344]:
# new delta result
an

array([[[[ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383],
         [ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383]],

        [[ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383],
         [ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383]]]])

In [373]:
gb = an.sum(axis=(0, 2, 3))
gW = np.tensordot(an, col, ([0, 2, 3], [0, 4, 5]))

nextEp = np.tensordot(W, an, (0, 1))

In [302]:
gb

array([ 0.16608272,  0.16608272])

In [323]:
gW

array([[[[ 0.17238397,  0.17238397],
         [ 0.33846668,  0.33846668]]],


       [[[ 0.17238397,  0.17238397],
         [ 0.33846668,  0.33846668]]]])

In [374]:
nextEp.shape

(1, 2, 2, 1, 4, 4)

In [377]:
nextEp

array([[[[[[ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
           [ 0.00039383,  0.00039383,  0.00039383,  0.00039383],
           [ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
           [ 0.00039383,  0.00039383,  0.00039383,  0.00039383]],

          [[ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
           [ 0.00039383,  0.00039383,  0.00039383,  0.00039383],
           [ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
           [ 0.00039383,  0.00039383,  0.00039383,  0.00039383]]],


         [[[ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
           [ 0.00039383,  0.00039383,  0.00039383,  0.00039383],
           [ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
           [ 0.00039383,  0.00039383,  0.00039383,  0.00039383]],

          [[ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
           [ 0.00039383,  0.00039383,  0.00039383,  0.00039383],
           [ 0.02036651,  0.02036651,  0.02036651,  0.02036651],
           [ 0.00

In [378]:
nextEp = np.rollaxis(nextEp, 3)

h, w = new_val.shape[2:]

In [380]:
nextEp.shape

(2, 1, 1, 2, 4, 4)

In [337]:
sy = sx = 2
ph = pw = 2
r1 = col2im(nextEp, sy, sx, ph, pw, h, w)

In [338]:
r1 

array([[[[ 0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.00039383,
           0.00039383,  0.        ,  0.        ],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.00039383,
           0.00039383,  0.        ,  0.        ],
         [ 0.02036651,  0.02036651,  0.02036651,  0.02036651,  0.02036651,
           0.02036651,  0.        ,  0.        ],
         [ 0.02036651,  0.02036651,  0.02036651,  0.02036651,  0.02036651,
           0.02036651,  0.        ,  0.        ],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.00039383,
           0.00039383,  0.        ,  0.        ],
         [ 0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.00039383,
           0.00039383,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ]]]

In [339]:
r2.ravel()

array([ 0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.00039383,
        0.00039383,  0.        ,  0.        ,  0.00039383,  0.00039383,
        0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.        ,
        0.        ,  0.02036651,  0.02036651,  0.02036651,  0.02036651,
        0.02036651,  0.02036651,  0.        ,  0.        ,  0.02036651,
        0.02036651,  0.02036651,  0.02036651,  0.02036651,  0.02036651,
        0.        ,  0.        ,  0.00039383,  0.00039383,  0.00039383,
        0.00039383,  0.00039383,  0.00039383,  0.        ,  0.        ,
        0.00039383,  0.00039383,  0.00039383,  0.00039383,  0.00039383,
        0.00039383,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ])

In [340]:
np.array_equal(r1,r2)

True

*old delta stuff*

In [162]:
deriv_z = deriv_sigmoid(z2)

In [164]:
# Pulled from the site
activationD = np.array(
[[[[0.02,0.02,0.02,0.02],
 [0.00,0.00,0.00,0.00],
 [0.02,0.02,0.02,0.02],
 [0.00,0.00,0.00,0.00]],
  [[0.02,0.02,0.02,0.02],
 [0.00,0.00,0.00,0.00],
 [0.02,0.02,0.02,0.02],
 [0.00,0.00,0.00,0.00]]]])

In [171]:
activationD

array([[[[ 0.02,  0.02,  0.02,  0.02],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.02,  0.02,  0.02,  0.02],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.02,  0.02,  0.02,  0.02],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.02,  0.02,  0.02,  0.02],
         [ 0.  ,  0.  ,  0.  ,  0.  ]]]])

In [163]:
deriv_z

array([[[[-12., -12., -12., -12.],
         [-56., -56., -56., -56.],
         [-12., -12., -12., -12.],
         [-56., -56., -56., -56.]],

        [[-12., -12., -12., -12.],
         [-56., -56., -56., -56.],
         [-12., -12., -12., -12.],
         [-56., -56., -56., -56.]]]])

In [151]:
deriv_z.shape

(1, 2, 4, 4)

In [152]:
epsilon = np.ones((1,2,4,4))

In [153]:
epsilon.shape

(1, 2, 4, 4)

In [154]:
delta2 = np.dot(epsilon, deriv_z)

In [155]:
epsilon

array([[[[ 1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.]],

        [[ 1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.]]]])

In [156]:
deriv_z

array([[[[-12., -12., -12., -12.],
         [-56., -56., -56., -56.],
         [-12., -12., -12., -12.],
         [-56., -56., -56., -56.]],

        [[-12., -12., -12., -12.],
         [-56., -56., -56., -56.],
         [-12., -12., -12., -12.],
         [-56., -56., -56., -56.]]]])

In [157]:
delta2.shape

(1, 2, 4, 1, 2, 4)

In [173]:
# pulled acivation from online - need to validate
delta3 = epsilon * activationD

In [174]:
delta3

array([[[[ 0.02,  0.02,  0.02,  0.02],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.02,  0.02,  0.02,  0.02],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.02,  0.02,  0.02,  0.02],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.02,  0.02,  0.02,  0.02],
         [ 0.  ,  0.  ,  0.  ,  0.  ]]]])

In [175]:
delta3.ravel()

array([ 0.02,  0.02,  0.02,  0.02,  0.  ,  0.  ,  0.  ,  0.  ,  0.02,
        0.02,  0.02,  0.02,  0.  ,  0.  ,  0.  ,  0.  ,  0.02,  0.02,
        0.02,  0.02,  0.  ,  0.  ,  0.  ,  0.  ,  0.02,  0.02,  0.02,
        0.02,  0.  ,  0.  ,  0.  ,  0.  ])

In [535]:
wG = np.tensordot(delta, col, ([0, 2, 3],[0, 4, 5]))

In [502]:
delta

array([[[[-544., -544.]],

        [[-544., -544.]]]])

In [536]:
wG.shape

(2, 1, 2, 2)

In [538]:
wG.ravel()

array([-1440., -1440., -1984., -1984., -1440., -1440., -1984., -1984.])

In [548]:
delta[0].shape

(2, 4, 4)

In [546]:
W.shape

(2, 1, 2, 2)

In [560]:
# nextEpsilon
r = np.tensordot(W,delta,([0,1]))

In [562]:
r = np.rollaxis(r,3)

In [579]:
r.shape

(1, 1, 2, 2, 4, 4)

In [564]:
r.ravel()

array([-12., -12., -12., -12., -56., -56., -56., -56., -12., -12., -12.,
       -12., -56., -56., -56., -56., -12., -12., -12., -12., -56., -56.,
       -56., -56., -12., -12., -12., -12., -56., -56., -56., -56., -12.,
       -12., -12., -12., -56., -56., -56., -56., -12., -12., -12., -12.,
       -56., -56., -56., -56., -12., -12., -12., -12., -56., -56., -56.,
       -56., -12., -12., -12., -12., -56., -56., -56., -56.])

In [566]:
def col2im(col, sy, sx, ph, pw, h, w):
    n, c, kh, kw, out_h, out_w = col.shape

    img = np.zeros((n, c, h + 2 * ph + sy - 1, w + 2 * pw + sx - 1), dtype=col.dtype)
    for i in six.moves.range(kh):
        i_lim = i + sy * out_h
        for j in six.moves.range(kw):
            j_lim = j + sx * out_w
            img[:, :, i:i_lim:sy, j:j_lim:sx] += col[:, :, i, j, :, :]

    return img[:, :, ph:h + ph, pw:w + pw]

In [575]:
reshapedEp = col2im(r, sy, sx, ph, pw, h, w)

In [576]:
reshapedEp.shape

(1, 1, 8, 8)

In [578]:
reshapedEp.ravel()

array([-12., -12., -12., -12., -12., -12., -12., -12., -12., -12., -12.,
       -12., -12., -12., -12., -12., -56., -56., -56., -56., -56., -56.,
       -56., -56., -56., -56., -56., -56., -56., -56., -56., -56., -12.,
       -12., -12., -12., -12., -12., -12., -12., -12., -12., -12., -12.,
       -12., -12., -12., -12., -56., -56., -56., -56., -56., -56., -56.,
       -56., -56., -56., -56., -56., -56., -56., -56., -56.])

### Checking activations & backprop - Subsampling

##### Activate

In [6]:
def im2col(img, kh, kw, sy, sx, ph, pw, pval=0, cover_all=False):
    n, c, h, w = img.shape
    featureMapWidth = (w-kw+pw*2)/sx + 1
    featureMapHeight = (h-kh+ph*2)/sy + 1 

    img = np.pad(img, ((0, 0), (0, 0), (ph, ph + sy - 1), (pw, pw + sx - 1)),
                    mode='constant', constant_values=(pval,))
    col = np.ndarray((n, c, kh, kw, featureMapHeight, featureMapWidth), dtype=img.dtype)

    for i in six.moves.range(kh):
        i_lim = i + sy * featureMapHeight
        for j in six.moves.range(kw):
            j_lim = j + sx * featureMapWidth
            col[:, :, i, j, :, :] = img[:, :, i:i_lim:sy, j:j_lim:sx]
    return col

In [10]:
def forward_max(x):
    kh=2
    kw=2
    ph = 0
    pw = 0
    sy = 2
    sx = 2
    col = im2col(x, kh, kw, sy, sx, ph, pw)
    n, c, kh, kw, out_h, out_w = col.shape
    col = col.reshape(n, c, kh * kw, out_h, out_w)

    # We select maximum twice, since the implementation using numpy.choose
    # hits its bug when kh * kw >= 32.
    indexes = col.argmax(axis=2)
    y = col.max(axis=2)
    return y

def forward_mean(x):
    kh=2
    kw=2
    ph = 0
    pw = 0
    sy = 2
    sx = 2
    col = im2col(x, kh, kw, sy, sx, ph, pw)
#     n, c, kh, kw, out_h, out_w = col.shape
#     col = col.reshape(n, c, kh * kw, out_h, out_w)

#     y = col.average(axis=2,3)
    return y

In [397]:
y2 = np.array([[[[ 1.,  1.,  3.,  7.],
                 [ 5.,  1.,  3.,  3.],
                 [ 2.,  2.,  8.,  4.],
                 [ 2.,  6.,  4.,  4.]],

                [[ 3.,  3.,  6.,  7.],
                 [ 4.,  4.,  6.,  7.],
                 [ 5.,  5.,  9.,  8.],
                 [ 4.,  4.,  9.,  8.]]]])

In [398]:
y2.shape

(1, 2, 4, 4)

In [443]:
forward_max(y2)

array([[[[ 5.,  7.],
         [ 6.,  8.]],

        [[ 4.,  7.],
         [ 5.,  9.]]]])

In [444]:
indexes

array([[[[2, 1],
         [3, 0]],

        [[2, 1],
         [0, 0]]]])

In [400]:
w = y2.shape[2]
kw=2
pw = 0
sx = 2

featureMapWidth = (w-kw+pw*2)/sx + 1

In [401]:
featureMapWidth

2

In [402]:
kh=2
kw=2
ph = 0
pw = 0
sy = 2
sx = 2

col = im2col(y2, kh, kw, sy, sx, ph, pw)

In [403]:
col

array([[[[[[ 1.,  3.],
           [ 2.,  8.]],

          [[ 1.,  7.],
           [ 2.,  4.]]],


         [[[ 5.,  3.],
           [ 2.,  4.]],

          [[ 1.,  3.],
           [ 6.,  4.]]]],



        [[[[ 3.,  6.],
           [ 5.,  9.]],

          [[ 3.,  7.],
           [ 5.,  8.]]],


         [[[ 4.,  6.],
           [ 4.,  9.]],

          [[ 4.,  7.],
           [ 4.,  8.]]]]]])

In [404]:
col.shape

(1, 2, 2, 2, 2, 2)

In [405]:
n, c, kh, kw, out_h, out_w = col.shape

In [406]:
col = col.reshape(n, c, kh * kw, out_h, out_w)


In [407]:
col.shape

(1, 2, 4, 2, 2)

In [408]:
indexes = col.argmax(axis=2)

In [409]:
indexes

array([[[[2, 1],
         [3, 0]],

        [[2, 1],
         [0, 0]]]])

In [410]:
indexes.shape

(1, 2, 2, 2)

In [411]:
y = col.max(axis=2)

In [412]:
y2

array([[[[ 1.,  1.,  3.,  7.],
         [ 5.,  1.,  3.,  3.],
         [ 2.,  2.,  8.,  4.],
         [ 2.,  6.,  4.,  4.]],

        [[ 3.,  3.,  6.,  7.],
         [ 4.,  4.,  6.,  7.],
         [ 5.,  5.,  9.,  8.],
         [ 4.,  4.,  9.,  8.]]]])

In [413]:
y

array([[[[ 5.,  7.],
         [ 6.,  8.]],

        [[ 4.,  7.],
         [ 5.,  9.]]]])

In [414]:
col.mean(axis=(2))

array([[[[ 2. ,  4. ],
         [ 3. ,  5. ]],

        [[ 3.5,  6.5],
         [ 4.5,  8.5]]]])

In [415]:
forward_mean(y2)

array([[[[ 5.,  7.],
         [ 6.,  8.]],

        [[ 4.,  7.],
         [ 5.,  9.]]]])

In [416]:
forward_mean(y2).shape

(1, 2, 2, 2)

##### backprop

In [417]:
def col2im(col, sy, sx, ph, pw, h, w):
    n, c, kh, kw, out_h, out_w = col.shape

    img = np.zeros((n, c, h + 2 * ph + sy - 1, w + 2 * pw + sx - 1), dtype=col.dtype)
    for i in six.moves.range(kh):
        i_lim = i + sy * out_h
        for j in six.moves.range(kw):
            j_lim = j + sx * out_w
            img[:, :, i:i_lim:sy, j:j_lim:sx] += col[:, :, i, j, :, :]

    return img[:, :, ph:h + ph, pw:w + pw]

In [389]:
def backward_max(x, gy):
    n, c, out_h, out_w = gy.shape
    h, w = x.shape[2:]
    kh = 2
    kw = 2
    gcol = np.zeros((n, c, kh, kw, out_h, out_w), dtype=np.float32)

    # TODO(beam2d): Make it fast
    gcol_r = np.rollaxis(gcol.reshape(n, c, -1, out_h, out_w), 2)
    for i in np.ndindex(n, c, out_h, out_w):
        gcol_r[indexes[i]][i] = gy[i]

    gx = col2im(gcol, sy, sx, ph, pw, h, w)
    return gx

def backward_mean(x, gy):
#     n, c, out_h, out_w = gy.shape
    h, w = x.shape[2:]
    kh = 2
    kw = 2

    gcol = np.tile(gy[0][:,:,np.newaxis, np.newaxis], (1, 1, kh, kw, 1, 1))
    gx = col2im(gcol, sy, sx, ph, pw, h, w)
    gx /= (kh * kw)
    return gx

In [384]:
epsilon = np.ones((1,2,2,2))

In [385]:
epsilon.ravel()

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [386]:
y2

array([[[[ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.],
         [ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.]],

        [[ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.],
         [ 4.,  4.,  4.,  4.],
         [ 8.,  8.,  8.,  8.]]]])

In [418]:
indexes

array([[[[2, 1],
         [3, 0]],

        [[2, 1],
         [0, 0]]]])

In [447]:
backward_max(y2, epsilon)

array([[[[ 0.,  0.,  0.,  1.],
         [ 1.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.],
         [ 0.,  1.,  0.,  0.]],

        [[ 0.,  0.,  0.,  1.],
         [ 1.,  0.,  0.,  0.],
         [ 1.,  0.,  1.,  0.],
         [ 0.,  0.,  0.,  0.]]]], dtype=float32)

In [448]:
backward_max(y2, epsilon).shape

(1, 2, 4, 4)

In [449]:
n, c, out_h, out_w = epsilon.shape
h, w = y2.shape[2:]
kh = 2
kw = 2

gcol = np.zeros((n, c, kh, kw, out_h, out_w), dtype=np.float32)

In [450]:
gcol.shape

(1, 2, 2, 2, 2, 2)

In [451]:
gcol_r = np.rollaxis(gcol.reshape(n, c, -1, out_h, out_w), 2)

In [452]:
gcol_r.shape

(4, 1, 2, 2, 2)

In [458]:
gy.shape

(1, 2, 4, 4)

In [459]:
n, c, out_h, out_w = epsilon.shape
for i in np.ndindex(n, c, out_h, out_w):
    gcol_r[indexes[i]][i] = epsilon[i]

In [460]:
gcol_r.shape

(4, 1, 2, 2, 2)

In [462]:
gcol_r

array([[[[[ 0.,  0.],
          [ 0.,  1.]],

         [[ 0.,  0.],
          [ 1.,  1.]]]],



       [[[[ 0.,  1.],
          [ 0.,  0.]],

         [[ 0.,  1.],
          [ 0.,  0.]]]],



       [[[[ 1.,  0.],
          [ 0.,  0.]],

         [[ 1.,  0.],
          [ 0.,  0.]]]],



       [[[[ 0.,  0.],
          [ 1.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]]], dtype=float32)

In [428]:
epsilon2=np.array([[[[ 1.,  2.],
                     [ 3.,  4.]],

                    [[ 5.,  6.],
                     [ 7.,  8.]]]])

In [719]:
gcol_r.shape

(4, 1, 2, 2, 2)

In [755]:
out_w

2

In [446]:
for i in np.ndindex(n, c, out_h, out_w):
    print gcol_r[i]
    gcol_r[i][i] = epsilon2[i]
    print gcol_r[i]
    print epsilon2[i]
    print i

[ 1.  1.]


IndexError: too many indices for array

In [751]:
gcol_r

array([[[[[ 4.,  4.],
          [ 4.,  4.]],

         [[ 4.,  4.],
          [ 4.,  4.]]]],



       [[[[ 4.,  4.],
          [ 4.,  4.]],

         [[ 4.,  4.],
          [ 4.,  4.]]]],



       [[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]],



       [[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]]], dtype=float32)

In [749]:
gcol_r[i]

array([ 1.,  1.], dtype=float32)

In [739]:
gcol_r.shape

(4, 1, 2, 2, 2)

In [729]:
gcol_r.shape

(4, 1, 2, 2, 2)

In [743]:
gcol_r

array([[[[[ 1.,  1.],
          [ 2.,  2.]],

         [[ 3.,  3.],
          [ 4.,  4.]]]],



       [[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]],



       [[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]],



       [[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]]], dtype=float32)

In [731]:
indexes.shape

(1, 2, 2, 2)

In [15]:
epsilon2.shape

(1, 2, 2, 2)

In [20]:
epsilon2[0].shape

(2, 2, 2)

In [429]:
epsilon2[:,:,np.newaxis, np.newaxis]

array([[[[[[ 1.,  2.],
           [ 3.,  4.]]]],



        [[[[ 5.,  6.],
           [ 7.,  8.]]]]]])

In [430]:
v = epsilon2[0][:,:,np.newaxis, np.newaxis]

In [431]:
v.shape

(2, 2, 1, 1, 2)

In [432]:
v

array([[[[[ 1.,  2.]]],


        [[[ 3.,  4.]]]],



       [[[[ 5.,  6.]]],


        [[[ 7.,  8.]]]]])

In [773]:
np.tile(epsilon2[0][:,:,np.newaxis, np.newaxis], (1,1,kh,kw,1,1))

array([[[[[[ 1.,  2.]],

          [[ 1.,  2.]]],


         [[[ 3.,  4.]],

          [[ 3.,  4.]]],


         [[[ 1.,  2.]],

          [[ 1.,  2.]]],


         [[[ 3.,  4.]],

          [[ 3.,  4.]]]],



        [[[[ 5.,  6.]],

          [[ 5.,  6.]]],


         [[[ 7.,  8.]],

          [[ 7.,  8.]]],


         [[[ 5.,  6.]],

          [[ 5.,  6.]]],


         [[[ 7.,  8.]],

          [[ 7.,  8.]]]]]])

In [433]:
np.tile(epsilon2[0][:,:,np.newaxis, np.newaxis], (1,1,kh,kw,1,1)).shape

(1, 2, 4, 2, 1, 2)

In [442]:
# AVG - Reshape
(col2im(np.tile(epsilon2[0][:,:,np.newaxis, np.newaxis], (1,1,kh,kw,1,1)), sy, sx, ph, pw, h, w)/4)

array([[[[ 0.25,  0.25,  0.5 ,  0.5 ],
         [ 0.75,  0.75,  1.  ,  1.  ],
         [ 0.25,  0.25,  0.5 ,  0.5 ],
         [ 0.75,  0.75,  1.  ,  1.  ]],

        [[ 1.25,  1.25,  1.5 ,  1.5 ],
         [ 1.75,  1.75,  2.  ,  2.  ],
         [ 1.25,  1.25,  1.5 ,  1.5 ],
         [ 1.75,  1.75,  2.  ,  2.  ]]]])

In [463]:
(col2im(np.tile(epsilon2[0][:,:,np.newaxis, np.newaxis], (1,1,kh,kw,1,1)), sy, sx, ph, pw, h, w)/4).shape

(1, 2, 4, 4)

In [787]:
# missing col2im for shaping
((np.tile(epsilon2[0][:,:,np.newaxis, np.newaxis], (1,1,kh,kw,1,1)))/(kh*kw)).shape

(1, 2, 4, 2, 1, 2)

In [778]:
epsilon2[0][:,:,np.newaxis, np.newaxis].shape

(2, 2, 1, 1, 2)

In [779]:
epsilon2[:,:,np.newaxis, np.newaxis].shape

(1, 2, 1, 1, 2, 2)

In [780]:
epsilon2.shape

(1, 2, 2, 2)

In [32]:
backward_mean(y2,epsilon2).shape

(1, 2, 4, 4)

In [789]:
epsilon2.ravel()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.])

In [792]:
W.shape

(2, 1, 2, 2)

In [795]:
backward_mean(y2,W).shape

(1, 1, 4, 4)

In [794]:
y2.shape

(1, 2, 4, 4)

In [796]:
backward_max(y2,W).shape

IndexError: index 1 is out of bounds for axis 0 with size 1

In [40]:
tmp = np.array(
        [[[[3.00,3.00],
         [3.00,3.00],
         [3.00,3.00],
         [3.00,3.00]],
          [[7.00,7.00],
         [7.00,7.00],
         [7.00,7.00],
         [7.00,7.00]],
          [[3.00,3.00],
         [3.00,3.00],
         [3.00,3.00],
         [3.00,3.00]],
          [[7.00,7.00],
         [7.00,7.00],
         [7.00,7.00],
         [7.00,7.00]]]])

In [41]:
tmp.shape

(1, 4, 4, 2)

In [42]:
tmp2 = np.array([[[[3.00,3.00],
 [3.00,3.00],
 [3.00,3.00],
 [3.00,3.00]],
  [[7.00,7.00],
 [7.00,7.00],
 [7.00,7.00],
 [7.00,7.00]],
  [[3.00,3.00],
 [3.00,3.00],
 [3.00,3.00],
 [3.00,3.00]],
  [[7.00,7.00],
 [7.00,7.00],
 [7.00,7.00],
 [7.00,7.00]]]])

In [43]:
tmp2.shape

(1, 4, 4, 2)